# Anime Recommendation System
#### Alejandra Avitia-Davila, Crystal Feng, Simon Chau, and Tyler Lafond

In [94]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA

animes_df = pd.read_csv("animes.csv")
animes_df = animes_df.drop(columns=['synopsis', 'aired', 'episodes', 'members', 'img_url', 'link']).drop_duplicates()
profiles_df = pd.read_csv("profiles.csv")
profiles_df = profiles_df.drop(columns = ['gender', 'birthday', 'link']).drop_duplicates()
print(animes_df.head())
print(profiles_df.head())
print(len(profiles_df.index))
#profiles_df_2 = profiles_df.head(5)

     uid                             title  \
0  28891           Haikyuu!! Second Season   
1  23273           Shigatsu wa Kimi no Uso   
2  34599                     Made in Abyss   
3   5114  Fullmetal Alchemist: Brotherhood   
4  31758  Kizumonogatari III: Reiketsu-hen   

                                               genre  popularity  ranked  \
0  ['Comedy', 'Sports', 'Drama', 'School', 'Shoun...         141    25.0   
1  ['Drama', 'Music', 'Romance', 'School', 'Shoun...          28    24.0   
2  ['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...          98    23.0   
3  ['Action', 'Military', 'Adventure', 'Comedy', ...           4     1.0   
4   ['Action', 'Mystery', 'Supernatural', 'Vampire']         502    22.0   

   score  
0   8.82  
1   8.83  
2   8.83  
3   9.23  
4   8.83  
           profile                                    favorites_anime
0   DesolatePsyche  ['33352', '25013', '5530', '33674', '1482', '2...
1        baekbeans  ['11061', '31964', '853', '20583', '918'

***Defining functions:***
- genre_list(anime_uid)
- anime_list(profile)

In [92]:
def genre_list(anime_uid):
    if(anime_uid == ""): return []
    anime_uid = int(anime_uid)
    if(len(animes_df.loc[animes_df["uid"] == anime_uid]) == 0): return []
    raw_genres = animes_df.loc[animes_df["uid"] == anime_uid]["genre"].array[0]
    genres = raw_genres.strip("][").replace("'", "").split(", ")
    for genre in genres:
        genre = genre.replace(" ", "")
    return genres

#print(genre_list(33352))

def anime_list(profile):
    if(profile > len(profiles_df.index)) : return []
    raw_favorites = profiles_df.iloc[profile]["favorites_anime"]
    favorites = raw_favorites.strip("][").replace("'", "").split(", ")
    for anime in favorites:
        anime = anime.replace("'", "")
    return favorites
    
#print(anime_list(0))

### Computing frequencies for profiles

In [93]:
#commented out and saved to csv since this process takes a while

#df_profile_freq = pd.DataFrame() #index is same as profile_df_index
#for profile in profiles_df.index:
#    favorites = anime_list(profile)
#    genres = []
#    for anime in favorites:
#        genres += genre_list(anime)
#    genre_series = pd.Series(genres)
#    num_genres = len(genres)
#    genre_frequencies = genre_series.value_counts() / num_genres
#    df_profile_freq = df_profile_freq.append(genre_frequencies, ignore_index=True)
#df_profile_freq = df_profile_freq.drop(columns=[""])
#df_profile_freq = df_profile_freq.fillna(0)
#df_profile_freq.to_csv("profile_genres.csv", index=False)